In [8]:
AGENT_NAME = '20 bin PPO 500 results/default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'

from stable_baselines3 import PPO
import numpy as np
import KBMproject.utilities as utils

In [4]:
SIZE = 100_000

##### Optimize _init_sample() function by vectorizing loop
we find that the vectorized code is roughly 10x faster than producing the same number of samples in a loop

In [3]:
%%timeit
for _ in range(SIZE):
   out = np.random.uniform(size=(1,31))

290 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
out2 = np.random.uniform(size=(SIZE,)+(1,31))

23.2 ms ± 479 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
from art.estimators.classification import PyTorchClassifier as classifier
from torch.nn import CrossEntropyLoss

agent = PPO.load(path=f"{AGENT_NAME}")

policy = classifier(
                    model=utils.extract_actor(agent),
                    loss=CrossEntropyLoss(), 
                    nb_classes=agent.action_space[0].n,
                    input_shape=agent.observation_space.shape,
                    device_type='gpu',
                    clip_values = (agent.observation_space.low.min(),agent.observation_space.high.max())
                    )

In [5]:
%%timeit
for _ in range(SIZE):
   rand_obs_loop = np.random.uniform(size=(1,31)).astype('float32')
   rand_preds_loop = np.argmax(policy.predict(rand_obs_loop), axis=1)

1min 35s ± 3.19 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
rand_obs_vec = np.random.uniform(size=(SIZE,)+(1,31)).astype('float32')
rand_preds_vec = np.argmax(policy.predict(rand_obs_vec), axis=2)
target_samples = rand_obs_vec[rand_preds_vec == np.array([10])] #starting points of the correct class
#check if target samples is empty -> attack failed
target_norms = np.linalg.norm(target_samples - rand_obs_vec[0], ord=2) #using an observation as a placeholder for the initial sample
starting_point = target_samples[np.argmin(target_norms)]# find the random obs colsest to the initial sample

829 ms ± 3.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
from KBMproject.mybb import BrendelBethgeAttack as mybba